In [1]:
#LAB 8 - Gerardo Garza Sada

import polars as pl

# Load the diamonds dataset
print("Loading diamonds dataset...")
df = pl.read_csv('https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/diamond.csv')
print("Dataset loaded successfully!")

Loading diamonds dataset...
Dataset loaded successfully!


In [2]:
#Question 1

#How extensive is our inventory database?

print(f"Total records (rows): {df.height:,}")
print(f"Total attributes (columns): {df.width}")
print("Column names:", df.columns)

Total records (rows): 6,000
Total attributes (columns): 8
Column names: ['Carat Weight', 'Cut', 'Color', 'Clarity', 'Polish', 'Symmetry', 'Report', 'Price']


In [3]:
#Question 2

#What information do we track about each diamond?

print ("columns tracked per diamond:")
print (df.head(10))

columns tracked per diamond:
shape: (10, 8)
┌──────────────┬───────────┬───────┬─────────┬────────┬──────────┬────────┬───────┐
│ Carat Weight ┆ Cut       ┆ Color ┆ Clarity ┆ Polish ┆ Symmetry ┆ Report ┆ Price │
│ ---          ┆ ---       ┆ ---   ┆ ---     ┆ ---    ┆ ---      ┆ ---    ┆ ---   │
│ f64          ┆ str       ┆ str   ┆ str     ┆ str    ┆ str      ┆ str    ┆ i64   │
╞══════════════╪═══════════╪═══════╪═════════╪════════╪══════════╪════════╪═══════╡
│ 1.1          ┆ Ideal     ┆ H     ┆ SI1     ┆ VG     ┆ EX       ┆ GIA    ┆ 5169  │
│ 0.83         ┆ Ideal     ┆ H     ┆ VS1     ┆ ID     ┆ ID       ┆ AGSL   ┆ 3470  │
│ 0.85         ┆ Ideal     ┆ H     ┆ SI1     ┆ EX     ┆ EX       ┆ GIA    ┆ 3183  │
│ 0.91         ┆ Ideal     ┆ E     ┆ SI1     ┆ VG     ┆ VG       ┆ GIA    ┆ 4370  │
│ 0.83         ┆ Ideal     ┆ G     ┆ SI1     ┆ EX     ┆ EX       ┆ GIA    ┆ 3171  │
│ 1.53         ┆ Ideal     ┆ E     ┆ SI1     ┆ ID     ┆ ID       ┆ AGSL   ┆ 12791 │
│ 1.0          ┆ Very Good ┆ D  

In [ ]:
#Question 3

#When I look at one line in this database, what am I looking at?

#Explanation (answer) = It depends, but for example, lets say we take a look at the first row of the data set, we are looking
# at one specific diamond inside all the data. So loking at the firt one, we can see different characteristics, such as:
# the cut, in this case is an ideal cut, we can also se the color, which is H. The clarity is SI1, the polish of the stones is
# VG, the symetry is EX, report would be GIA, and the cost of that diamond is $5,169.

In [5]:
#Question 4

#We're considering expanding our storage facilities. How many total carats do we have in inventory? I need to know if our vaults can handle the weight.

total_carats = df ["Carat Weight"].sum()
print (f"total carats in the inventory: {total_carats:,.2f}")

total carats in the inventory: 8,007.12


In [7]:
#Question 5

#Our new marketing campaign claims we have 'the widest variety of cuts in the industry.' Can you verify this?

unique_cuts = df ["Cut"].unique()
cut_counts = df ["Cut"].value_counts()
print ("Unique Cuts:")
print (unique_cuts)
print ("\nNumber Cut Counts:")
print (cut_counts)

Unique Cuts:
shape: (5,)
Series: 'Cut' [str]
[
	"Very Good"
	"Ideal"
	"Fair"
	"Good"
	"Signature-Ideal"
]

Number Cut Counts:
shape: (5, 2)
┌─────────────────┬───────┐
│ Cut             ┆ count │
│ ---             ┆ ---   │
│ str             ┆ u32   │
╞═════════════════╪═══════╡
│ Signature-Ideal ┆ 253   │
│ Fair            ┆ 129   │
│ Good            ┆ 708   │
│ Ideal           ┆ 2482  │
│ Very Good       ┆ 2428  │
└─────────────────┴───────┘


In [10]:
#Question 6

#I'm meeting with our insurance company. They need to know our price exposure - what's our most and least valuable diamond?

max_price = df.select(pl.col("Price").max()).item()
print(f"Highest price: ${max_price:,.2f}")

min_price = df.select(pl.col("Price").min()).item()
print(f"Lowest price: ${min_price:,.2f}")

Highest price: $101,561.00
Lowest price: $2,184.00


In [15]:
#Question 7

#Which color grades should we focus on in our premium line? I need typical prices across our color spectrum.

color_price = df.group_by(pl.col("Color")).agg(pl.col("Price").mean().alias("avg_price")).sort("avg_price", descending=True)
print("Average price by color grade (from most to least expensive):")
print(color_price)

#My answer = they should focus on the color D diamonds, because those are the most expensive ones.

Average price by color grade (from most to least expensive):
shape: (6, 2)
┌───────┬──────────────┐
│ Color ┆ avg_price    │
│ ---   ┆ ---          │
│ str   ┆ f64          │
╞═══════╪══════════════╡
│ D     ┆ 15255.783661 │
│ F     ┆ 12712.241856 │
│ G     ┆ 12520.050633 │
│ E     ┆ 11539.190231 │
│ H     ┆ 10487.347544 │
│ I     ┆ 8989.636364  │
└───────┴──────────────┘


In [23]:
#Question 8

#We're thinking of specializing in certain clarity grades. For our investor deck, I need a complete breakdown showing: how many diamonds we have in each clarity grade, their typical price, and the total carats we're holding for each grade."

unique_count = df.select(pl.col("Clarity").n_unique()).item()
print(f"Number of unique clarity grades: {unique_count}")

unique_values = df.select(pl.col("Clarity").unique().sort()).to_series().to_list()
print(f"The clarity grades are: {unique_values}")

clarity_analysis = (
    df.group_by("Clarity")
      .agg(
          pl.len().alias("num_diamonds"),
          pl.col("Price").mean().alias("avg_price"),
          pl.col("Carat Weight").sum().alias("total_carats")
      )
      .sort("avg_price", descending=True)
)
print("Clarity diamonds analysis:")
print(clarity_analysis)

Number of unique clarity grades: 7
The clarity grades are: ['FL', 'IF', 'SI1', 'VS1', 'VS2', 'VVS1', 'VVS2']
Clarity diamonds analysis:
shape: (7, 4)
┌─────────┬──────────────┬──────────────┬──────────────┐
│ Clarity ┆ num_diamonds ┆ avg_price    ┆ total_carats │
│ ---     ┆ ---          ┆ ---          ┆ ---          │
│ str     ┆ u32          ┆ f64          ┆ f64          │
╞═════════╪══════════════╪══════════════╪══════════════╡
│ FL      ┆ 4            ┆ 63776.0      ┆ 7.87         │
│ IF      ┆ 219          ┆ 22105.844749 ┆ 316.0        │
│ VVS1    ┆ 285          ┆ 16845.680702 ┆ 389.43       │
│ VVS2    ┆ 666          ┆ 14142.177177 ┆ 876.58       │
│ VS1     ┆ 1192         ┆ 13694.113255 ┆ 1682.74      │
│ VS2     ┆ 1575         ┆ 11809.053333 ┆ 2170.81      │
│ SI1     ┆ 2059         ┆ 8018.864012  ┆ 2563.69      │
└─────────┴──────────────┴──────────────┴──────────────┘


In [28]:
#Question 9

#My luxury competitor claims they have the best prices for every cut-color combination. Show me our pricing structure across these combinations - I want to see where we're competitive

result = df.group_by(["Cut", "Color"]).agg(
    pl.col("Price").mean().alias("avg_price"),
    pl.len().alias("count")
).sort(["Cut", "Color"])

print("NEW GRAIN: one row per cut-color combination")
print(f"Number of unique combinations: {len(result)}")
result.head(10)


NEW GRAIN: one row per cut-color combination
Number of unique combinations: 30


Cut,Color,avg_price,count
str,str,f64,u32
"""Fair""","""D""",6058.25,12
"""Fair""","""E""",5370.625,32
"""Fair""","""F""",6063.625,24
"""Fair""","""G""",7345.52381,21
"""Fair""","""H""",5908.5,24
"""Fair""","""I""",4573.1875,16
"""Good""","""D""",10058.716216,74
"""Good""","""E""",8969.545455,110
"""Good""","""F""",9274.007519,133


In [38]:
#Question 10

#URGENT - "Our biggest client is interested in purchasing entire inventory segments. They asked: 'Which color-cut combinations represent your highest total inventory value?' Show me the top 5 most valuable inventory segments by total worth, and include how many diamonds are in each segment."


top_valuable_segments = (
    df.group_by(["Cut", "Color"])
      .agg(
          pl.col("Price").sum().alias("total_value"),
          pl.len().alias("num_diamonds")
      )
      .sort("total_value", descending=True)
      .head(5)
)

print("Top 5 most expensive color/cut diamonds in the inventory segments:")
print(top_valuable_segments)


Top 5 most expensive color/cut diamonds in the inventory segments:
shape: (5, 4)
┌───────────┬───────┬─────────────┬──────────────┐
│ Cut       ┆ Color ┆ total_value ┆ num_diamonds │
│ ---       ┆ ---   ┆ ---         ┆ ---          │
│ str       ┆ str   ┆ i64         ┆ u32          │
╞═══════════╪═══════╪═════════════╪══════════════╡
│ Ideal     ┆ G     ┆ 9363514     ┆ 690          │
│ Very Good ┆ G     ┆ 7140620     ┆ 578          │
│ Very Good ┆ F     ┆ 5648327     ┆ 455          │
│ Ideal     ┆ F     ┆ 5346782     ┆ 363          │
│ Ideal     ┆ H     ┆ 5279687     ┆ 458          │
└───────────┴───────┴─────────────┴──────────────┘
